A demo notebook showing the flow of the algorithm (picking a ) 

In [1]:
import copy
import numpy as np
from scipy.sparse import csr_matrix
from tqdm import tqdm
import utils

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
path_neuron = './data/ol_columns.csv'
path_synapse = './data/ol_connections.csv'

In [3]:
df_neurons, N, K, T = utils.load_neurons(path_neuron)
d_neuron, d_type = utils.define_dict(df_neurons, N, T)
X, Y = utils.define_X_Y(df_neurons, d_type, N, K, T)
W = utils.load_synapses(path_synapse, d_neuron, N)
W_sparse = csr_matrix(W)

In [4]:
d_neurons_type = utils.neuron_list_per_type(Y)

In [5]:
f_curr= utils.f(X, W_sparse)  # starting value of the objective function
f_curr

1381915.0

In [6]:
def process_epsilon(X, W, W_sparse, f_curr, d_neurons_type, eps, T, seed, max_iter=200):
    """
    Subprocess to compute the best assignment for current seed and epsilon. 

    Args:
        X (np.array): neuron-column assigment
        W (np.array): synaptic connectivity matrix
        W_sparse (csr_matrix): sparse matrix version of W
        f_curr (float): current value of the objective function
        d_neurons_type (dict): cell type index -> list of neuron indices that belong to that cell type
        eps (float): epsilon for random selection
        T (int): number of types
        seed (int): random seed
        max_iter (int): maximum iterations, default is 200
    
    Returns:
        f_history (list, int): history of objective functions
        diff_history (list, int): history of increase in objective functions
        X_max (np.array): assignment matrix with highest objective function
        f_max (int): highest objective function
        eps (float): epsilon
        seed (int): random seed
    """
    
    np.random.seed(seed)
    f_history = []
    diff_history = []
    iteration = 0
    X_max = X
    f_max = f_curr
    while True:
        diff_per_type = []
        X_new_per_type = []

        for t in tqdm(range(T)):
            diff, X_new = utils.diff_given_type(X, W, W_sparse, t, f_curr, d_neurons_type)
            diff_per_type.append(diff)
            X_new_per_type.append(X_new)
        diff_history.append(dict(zip(range(T), diff_per_type)))
        
        # perform the reassignment
        max_diff = max(diff_per_type)
        print(max_diff)
        if np.random.uniform(0, 1) < eps:
            type_idx = np.random.choice(range(T))
        else: 
            type_idx = diff_per_type.index(max(diff_per_type))  # cell type that has the max improvement in the objective function

        X = copy.deepcopy(X_new_per_type[type_idx])
        f_curr = utils.f(X, W_sparse)  # update the current objective function
        print(f_curr)

        if f_curr > f_max:
            f_max = f_curr
            X_max = X
        f_history.append(f_curr)
        iteration += 1
        if iteration >= max_iter or max_diff == 0:
            break
    return f_history, diff_history, X_max, f_max, eps, seed

In [7]:
seed = 440 # test a single seed with different epsilons
maximum_iter = 1 # set to 1 for code testing
eps = 0.95 # used for our final result

In [8]:
# solve assignment problem
f_history, diff_history, X_max,f_max, eps,seed = process_epsilon(X, W, W_sparse, f_curr, d_neurons_type, eps, T, seed, max_iter=maximum_iter) 

100%|██████████| 31/31 [01:25<00:00,  2.77s/it]

1553.0
1381963.0
